# Testing

```source: this key separates the various keys found in the table in Sources. Here's the set of sources with their corresponding value name:```
```
'https://aipulse.org'
'ebook'
'https://qualiacomputing.com'
'alignment forum'
'lesswrong'
'manual'
'arxiv'
'https://deepmindsafetyresearch.medium.com/'
'waitbutwhy.com'
'GitHub'
'https://aiimpacts.org'
'arbital.com'
'carado.moe'
'nonarxiv_papers'
'https://vkrakovna.wordpress.com'
'https://jsteinhardt.wordpress.com'
'audio-transcripts'
'https://intelligence.org'
'youtube'
'reports'
'https://aisafety.camp'
'curriculum'
'https://www.yudkowsky.net'
'distill'
```

```...and this is how the arxiv papers look like:```

```
{
    "source": "arxiv", # where the dataset comes from
    "source_type": "latex", # the type of file the data was original in
    "converted_with": "pandoc", # which tool we used to convert the data in .md format
    "paper_version": paper_id,
    "title": title,
    "authors": [str(x) for x in authors], # list of authors
    "date_published": date_published,
    "data_last_modified": data_last_modified,
    "url": url,
    "abstract": abstract,
    "author_comment": author_comment,
    "journal_ref": journal_ref,
    "doi": doi,
    "primary_category": primary_category,
    "categories": categories,
    "citation_level": citation_level, # (0 = curated alignment papers, 1 = citation of curated papers, 2 = citation of citation, etc.)
    "alignment_text": is_alignment_text, # 'pos' is maunally labeled as an alignment paper, 'unlabeled' if unlabeled
    "confidence_score": confidence_scores, # this is a confidence score obtained by using the SPECTER model to classify papers to add to the dataset
    "main_tex_filename": "main.tex", # the main latex file needed to convert the paper
    "text": "lots of text", # this is where you will grab the text contents of each entry in the dataset (in .md format)
    "bibliography_bbl": "string of bbl",
    "bibliography_bib": "string of bib", # more common to have bib than bbl
}
```

Useful links:

- Semantic Search OpenAI Cookbook: https://github.com/openai/openai-cookbook/blob/main/examples/Semantic_text_search_using_embeddings.ipynb

- Question-Answering OpenAI Cookbook: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

- Pinecone: https://app.pinecone.io

- Retrieval Enhanced Generative Question Answering with Pinecone: https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/pinecone/Gen_QA.ipynb

- Moderation: https://platform.openai.com/docs/guides/moderation/quickstart

- 5k Bounty: https://www.lesswrong.com/posts/SLRLuiuDykfTdmesK/speed-running-everyone-through-the-bad-alignement-bingo

- Handling rate-limits Cookbook: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

## Imports

In [2]:
import jsonlines
import numpy as np
from typing import List, Dict, Tuple, DefaultDict, Any
import re
import time
import random
import pickle
import openai
import concurrent.futures
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
import tiktoken

import config
from pathlib import Path
from collections import defaultdict

In [3]:
# FROM https://stackoverflow.com/a/31505798/16185542
# -*- coding: utf-8 -*-
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = text.replace("?!", "?")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")

    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    
    if sentences == []:
        sentences = [text.strip()]
    return sentences


class TokenSplitter:
    """splits text into blocks of tokens according to chatgpt's tokenizer"""
    def __init__(self, min_tokens: int = 500, max_tokens: int = 750):
        self.encoding = tiktoken.get_encoding("cl100k_base")
        self.min_tokens = min_tokens
        self.max_tokens = max_tokens
        self.blocks = []
        self.signature = "{url, title, author} unknown"
        


    def _text_splitter(self, text: str) -> List[str]:
        """splits text into blocks of tokens according to chatgpt's tokenizer"""
        # Do not call this function outside of split()      
        
        enc = self.encoding.encode # takes a string and returns a list of ints (tokens)
        dec = self.encoding.decode # takes a list of ints (tokens) and returns a string
        tok_len = lambda x: len(enc(x)) # length of a string in tokens

        max_tokens = self.max_tokens - tok_len(self.signature) - 10 # 10 to be safe
        assert max_tokens > 0, "max_tokens is too small for the signature"
        
        min_tokens = self.min_tokens - tok_len(self.signature) - 10 # 10 to be safe
        assert min_tokens > 0, "min_tokens is too small for the signature"

        current_block = ""
        paragraphs = text.split("\n\n")
        for paragraph in paragraphs:
            sentences = split_into_sentences(paragraph)
            if current_block != "":
                current_block += "\n\n"

            for sentence in sentences:
                potential_new_block = current_block + " " + sentence
                
                if tok_len(potential_new_block) <= max_tokens:
                    current_block = potential_new_block
                
                else:
                    self.blocks.append(current_block)
                    if tok_len(sentence) < max_tokens:
                        current_block = sentence
                    else:
                        self.blocks.append(dec(enc(sentence)[:max_tokens]))
                        current_block = ""
            
            if tok_len(current_block) > min_tokens:
                self.blocks.append(current_block)
                current_block = ""

        if current_block != "":
            if len(self.blocks) == 0:
                self.blocks.append(current_block)
                return
            latest_block = self.blocks[-1]
            len_cur_block = tok_len(current_block)
            latest_plus_current = latest_block + current_block

            if len_cur_block > min_tokens:
                self.blocks.append(current_block)
            
            else:
                #select the last self.max_tokens tokens from the latest block
                last_block = dec(enc(latest_plus_current)[-max_tokens:])
                self.blocks.append(last_block)

        
    
    def split(self, text: str, signature: str) -> List[str]:
        self.signature = signature
        self._text_splitter(text)
        blocks = self.blocks
        self.blocks = []
        self.signature = "{url, title, author} unknown"
        
        # check all block elements are strings
        assert all([isinstance(block, str) for block in blocks]), "block elements are not strings"

        output = [f"{block}\n - {signature}" for block in blocks]
        #check all output elements are strings
        assert all([isinstance(block, str) for block in output]), "output elements are not strings"

        return output

## Constants

In [11]:
LEN_EMBEDDINGS = 1536

COMPLETIONS_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

openai.api_key = config.OPENAI_API_KEY


project_path = Path.cwd().parent # .cwd() returns the current working directory, 
# but in a notebook it is the directory of the notebook. So we need to go up two levels.
PATH_TO_DATA = project_path / "src" / "data" / "alignment_texts.jsonl" # Path to the dataset .jsonl file.
PATH_TO_EMBEDDINGS = project_path / "src" / "data" / "embeddings.npy" # Path to the saved embeddings (.npy) file.
PATH_TO_DATASET = project_path / "src" / "data" / "dataset.pkl" # Path to the saved dataset (.pkl) file.

## Helpers

In [6]:
class MissingDataException(Exception):
    pass

## Dataset Class

In [129]:
error_count_dict = {
    "Entry has no source.": 0,
    "Entry has no title.": 0,
    "Entry has no text.": 0,
    "Entry has no URL.": 0,
    "Entry has wrong citation level.": 0
}

class Dataset:
    def __init__(self,
            jsonl_data_path: str,  # Path to the dataset .jsonl file.
            custom_sources: List[str] = None,  # List of sources to include, like "alignment forum", "lesswrong", "arxiv",etc.
            rate_limit_per_minute: int = 3_500,  # Rate limit for the OpenAI API.
            min_tokens_per_block: int = 400, # Minimum number of tokens per block.
            max_tokens_per_block: int = 600, # Maximum number of tokens per block.
            fraction_of_articles_to_use: float = 1.0,  # Fraction of articles to use. If 1.0, use all articles.
        ):
        self.jsonl_data_path = jsonl_data_path
        self.custom_sources = custom_sources
        self.rate_limit_per_minute = rate_limit_per_minute
        self.delay_in_seconds = 60.0 / self.rate_limit_per_minute
        self.fraction_of_articles_to_use = fraction_of_articles_to_use
        
        self.min_tokens_per_block = min_tokens_per_block  # for the text splitter
        self.max_tokens_per_block = max_tokens_per_block  # for the text splitter
        
        self.metadata: List[Tuple[str]] = []  # List of tuples, each containing the title of an article, its URL, and text. E.g.: [('title', 'url', 'text'), ...]
        self.embedding_strings: List[str] = []  # List of strings, each being a few paragraphs from a single article (not exceeding 1000 words).
        
        self.articles_count: DefaultDict[str, int] = defaultdict(int)  # Number of articles per source. E.g.: {'source1': 10, 'source2': 20, 'total': 30}

        if self.custom_sources is not None:
            for source in self.custom_sources:
                self.articles_count[source] = 0
        self.total_articles_count = 0
        
        self.total_char_count = 0
        self.total_word_count = 0
        self.total_sentence_count = 0
        self.total_block_count = 0
        
        self.sources_so_far: List[str] = []
        self.info_types: Dict[str, List[str]] = {}
    
    def extract_info_from_article(self, article: Dict[str, Any]) -> Tuple[str]:
        """
        This function extracts the title, author, date, URL, tags, and text from an article.
        
        Args:
            article (Dict[str, Any]): a dictionary containing the article's text and metadata.

        Returns:
            Tuple[str]: a tuple containing the title, author, date, URL, tags, and text of the article.
        """
        title = None
        author = None
        date_published = None
        url = None
        tags = None
        text = None
        
        # Get title
        if 'title' in article and 'book_title' in article and article['title']: title = article['title']
        elif 'book_title' in article and 'title' not in article and article['book_title']: 
            title = article['book_title']
            if title[-1] == '\n': title = title[:-1]
        elif 'title' in article and article['title']: 
            title = article['title']
            if title[-1] == '\n': title = title[:-1]
        else: title = None

        # Get author
        if 'author' in article and 'authors' in article and article['author']: author = article['author']
        elif 'authors' in article and article['authors']: author = article['authors']
        elif 'author' in article and article['author']: author = article['author']
        else: author = None

        # Get date published
        if 'date_published' in article and article['date_published'] and len(article['date_published']) >= 10: date_published = article['date_published'][:10]
        elif 'published' in article and article['published'] and len(article['published']) >= 16: date_published = article['published'][:16]
        else: date_published = None
            
        # Get URL
        if 'link' in article and article['link']: url = article['link']
        elif 'url' in article and article['url']: url = article['url']
        elif 'doi' in article and article['doi']: url = article['doi']
        else: url = None
            
        # Get tags
        if 'tags' in article and article['tags']:
            if type(article['tags']) == list: tags = ', '.join([val['term'] for val in article['tags']])
            elif type(article['tags']) == str: tags = article['tags']
            else: tags = None
        
        # Get text
        if 'text' in article and article['text']: text = article['text']
        else:
            raise MissingDataException(f"Entry has no text.")

        return (title, author, date_published, url, tags, text)
           
    def get_alignment_texts(self):
        text_splitter = TokenSplitter(self.min_tokens_per_block, self.max_tokens_per_block)
        with jsonlines.open(self.jsonl_data_path, "r") as reader:
            for entry in reader:
                try:
                    if 'source' not in entry: 
                        if 'url' in entry and entry['url'] == "https://www.cold-takes.com/": 
                            entry["source"] = "Cold Takes"
                        elif 'question' in entry and 'answer' in entry: 
                            entry["source"] = "printouts"
                            continue # for now, skip printouts
                        elif 'article_url' in entry and entry['article_url'] == "https://www.gwern.net":
                            entry["source"] = "gwern.net"
                        elif 'url' in entry and entry['url'] == "https://generative.ink/posts/":
                            entry["source"] = "generative.ink"
                        elif 'url' in entry and entry['url'][:24] == "https://greaterwrong.com":
                            entry["source"] = "greaterwrong.com"
                        else:
                            raise MissingDataException("Entry has no source.")
                    
                    random_number = random.random()
                    if random_number > self.fraction_of_articles_to_use:
                        continue
                    
                    # if we specified custom sources, only include articles from those sources
                    if (self.custom_sources is not None) and (entry['source'] not in self.custom_sources):
                        continue
                    self.articles_count[entry['source']] += 1
                    self.total_articles_count += 1
                    
                    # Get title, author, date, URL, tags, and text
                    title, author, date_published, url, tags, text = self.extract_info_from_article(entry)
                                                            
                    # Get signature
                    signature = ""
                    if title: signature += f"Title: {title}, "
                    if author: signature += f"Author: {author}, "
                    if date_published: signature += f"Date published: {date_published}, "
                    if url: signature += f"URL: {url}, "
                    # if tags: signature += f"Tags: {tags}, "  # Temporary decision to not include tags in the signature
                    if signature: signature = signature[:-2]
                    
                    # Add info to metadata and embedding strings
                    self.metadata.append((title, author, date_published, url, tags, text))
                    blocks = text_splitter.split(text, signature)
                    self.embedding_strings.extend(blocks)
                    
                    # Update counts
                    self.total_char_count += len(text)
                    self.total_word_count += len(text.split())
                    self.total_sentence_count += len(split_into_sentences(text))
                    self.total_block_count += len(blocks)
                
                except MissingDataException as e:
                    if str(e) not in error_count_dict:
                        error_count_dict[str(e)] = 0
                    error_count_dict[str(e)] += 1

    def get_embeddings(self):
        # Get an embedding for each text, with retries if necessary
        @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(5))
        def get_embedding_at_index(text: str, i: int, delay_in_seconds: float = 0) -> np.ndarray:
            time.sleep(delay_in_seconds)
            embedding = openai.Embedding.create(
                model=EMBEDDING_MODEL, 
                input=text
            )
            return i, embedding["data"][0]["embedding"]
        
        start = time.time()
        self.embeddings = np.zeros((len(self.embedding_strings), LEN_EMBEDDINGS))
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(get_embedding_at_index, text, i) for i, text in enumerate(self.embedding_strings)]
            num_completed = 0
            for future in concurrent.futures.as_completed(futures):
                i, embedding = future.result()
                self.embeddings[i] = embedding
                num_completed += 1
                if num_completed % 50 == 0:
                    print(f"Completed {num_completed}/{len(self.embedding_strings)} embeddings in {time.time() - start:.2f} seconds.")
        print(f"Completed {num_completed}/{len(self.embedding_strings)} embeddings in {time.time() - start:.2f} seconds.")
    
    def save_embeddings(self, path: str):
        np.save(path, self.embeddings)
        
    def load_embeddings(self, path: str):
        self.embeddings = np.load(path)
        
    def save_class(self, path: str):
        with open(path, 'wb') as f:
            pickle.dump(self, f)

In [131]:
# List of possible sources:
all_sources = ["https://aipulse.org", "ebook", "https://qualiacomputing.com", "alignment forum", "lesswrong", "manual", "arxiv", "https://deepmindsafetyresearch.medium.com", "waitbutwhy.com", "GitHub", "https://aiimpacts.org", "arbital.com", "carado.moe", "nonarxiv_papers", "https://vkrakovna.wordpress.com", "https://jsteinhardt.wordpress.com", "audio-transcripts", "https://intelligence.org", "youtube", "reports", "https://aisafety.camp", "curriculum", "https://www.yudkowsky.net", "distill",
               "Cold Takes", "printouts", "gwern.net", "generative.ink", "greaterwrong.com"] # These sources do not have a source field in the .jsonl file

# List of sources we are using for the test run:
custom_sources = [
    "https://aipulse.org", 
    # "ebook", 
    # "https://qualiacomputing.com", 
    # "alignment forum", 
    # "lesswrong", 
    "manual", 
    # "arxiv", 
    # "https://deepmindsafetyresearch.medium.com", 
    "waitbutwhy.com", 
    # "GitHub", 
    # "https://aiimpacts.org", 
    # "arbital.com", 
    # "carado.moe", 
    # "nonarxiv_papers", 
    "https://vkrakovna.wordpress.com", 
    "https://jsteinhardt.wordpress.com", 
    # "audio-transcripts", 
    # "https://intelligence.org", 
    # "youtube", 
    # "reports", 
    "https://aisafety.camp", 
    "curriculum", 
    "https://www.yudkowsky.net", 
    # "distill",
    # "Cold Takes",
    # "printouts",
    # "gwern.net",
    # "generative.ink",
    # "greaterwrong.com"
]

dataset = Dataset(
    jsonl_data_path=PATH_TO_DATA.resolve(), 
    custom_sources=custom_sources, 
    rate_limit_per_minute=3500, 
    min_tokens_per_block=200, max_tokens_per_block=350, 
    # fraction_of_articles_to_use=1/2000
)
dataset.get_alignment_texts()


# with open(PATH_TO_DATASET.resolve(), 'rb') as f:
#     dataset = pickle.load(f)

In [154]:
article_num = 73

example_article_metadata = dataset.metadata[article_num]
print(f"Title: {example_article_metadata[0]}")
print(f"Author: {example_article_metadata[1]}")
print(f"Date published: {example_article_metadata[2]}")
print(f"URL: {example_article_metadata[3]}")
print(f"Tags: {example_article_metadata[4]}")
print(f"Text: {example_article_metadata[5]}")


Title: Model Mis-specification and Inverse Reinforcement Learning
Author: jsteinhardt
Date published: Tue, 07 Feb 2017
URL: https://jsteinhardt.wordpress.com/2017/02/07/model-mis-specification-and-inverse-reinforcement-learning/
Tags: Locomotion
Text: Model Mis-specification and Inverse Reinforcement Learning

In my previous post, “Latent Variables and Model Mis-specification”, I argued that while machine learning is good at optimizing accuracy on observed signals, it has less to say about correctly inferring the values for unobserved variables in a model. In this post I’d like to focus in on a specific context for this: inverse reinforcement learning (Ng et al. 2000, Abeel et al. 2004, Ziebart et al. 2008, Ho et al 2016), where one observes the actions of an agent and wants to infer the preferences and beliefs that led to those actions. For this post, I am pleased to be joined by Owain Evans, who is an active researcher in this area and has co-authored an online book about building mo

In [133]:
print(f"Articles count: {len(dataset.metadata)}")
print(f"Num of each source: {dataset.articles_count}")
print(f"Num chars: {dataset.total_char_count}")
print(f"Num words: {dataset.total_word_count}")
print(f"Num sentences: {dataset.total_sentence_count}")
print(f"Num blocks: {dataset.total_block_count}")

Articles count: 140
Num of each source: defaultdict(<class 'int'>, {'https://aipulse.org': 23, 'manual': 1, 'waitbutwhy.com': 2, 'https://vkrakovna.wordpress.com': 43, 'https://jsteinhardt.wordpress.com': 39, 'https://aisafety.camp': 8, 'curriculum': 1, 'https://www.yudkowsky.net': 23})
Num chars: 2096402
Num words: 335374
Num sentences: 16648
Num blocks: 1818


In [139]:
dataset.get_embeddings()
dataset.save_embeddings(PATH_TO_EMBEDDINGS)

Completed 50/1818 embeddings in 1.75 seconds.
Completed 100/1818 embeddings in 2.48 seconds.
Completed 150/1818 embeddings in 3.32 seconds.
Completed 200/1818 embeddings in 4.09 seconds.
Completed 250/1818 embeddings in 4.94 seconds.
Completed 300/1818 embeddings in 5.83 seconds.
Completed 350/1818 embeddings in 6.62 seconds.
Completed 400/1818 embeddings in 7.43 seconds.
Completed 450/1818 embeddings in 8.23 seconds.
Completed 500/1818 embeddings in 8.90 seconds.
Completed 550/1818 embeddings in 9.60 seconds.
Completed 600/1818 embeddings in 10.39 seconds.
Completed 650/1818 embeddings in 11.25 seconds.
Completed 700/1818 embeddings in 11.92 seconds.
Completed 750/1818 embeddings in 12.73 seconds.
Completed 800/1818 embeddings in 13.69 seconds.
Completed 850/1818 embeddings in 14.88 seconds.
Completed 900/1818 embeddings in 16.02 seconds.
Completed 950/1818 embeddings in 17.34 seconds.
Completed 1000/1818 embeddings in 18.22 seconds.
Completed 1050/1818 embeddings in 19.32 seconds.
Co

In [140]:
dataset.save_class(PATH_TO_DATASET)

In [141]:
# for embed in dataset.embedding_strings:
#     print(embed)
#     print("\n"*4)

In [142]:
"""
TODO:
Add a moderation call to not be prompt-hacked: https://platform.openai.com/docs/guides/moderation/quickstart
"""

class AlignmentSearch:
    def __init__(self,
            dataset: Dataset,  # Dataset object containing the data.
        ):
        self.metadataset = dataset
    
    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(4))
    def get_embedding(self, text: str) -> np.ndarray:
        result = openai.Embedding.create(model=EMBEDDING_MODEL, input=text)
        print(f"Embedding created for query: {text}")
        return result["data"][0]["embedding"]
        # except openai.RateLimitError as e:
        #     print("Rate limit exceeded. Retrying in 30 seconds.")
        #     time.sleep(30)
        #     raise e
    
    def get_top_k(self, query: str, k: int=10) -> List[str]:
        # Receives a query (str) and returns the top k blocks that are most semantically similar to the query.
        # Each tuple contains the title of an article, its URL, and text.
        query_embedding = self.get_embedding(query)
        similarities = np.dot(self.metadataset.embeddings, query_embedding)
        top_k_indices = np.argsort(similarities)[::-1][:k]
        top_k = [self.metadataset.embedding_strings[i] for i in top_k_indices]
        return top_k
    
    def limit_tokens(self, text: str, max_tokens: int, encoding_name: str = "cl100k_base") -> str:
        encoding = tiktoken.get_encoding(encoding_name)
        tokens = encoding.encode(text)[:max_tokens]
        return encoding.decode(tokens)
    
    def construct_messages(self, question: str, blocks: List[str] = None, mode: str = "balanced") -> str:
        # Receives a question (str) and a list of blocks and returns a prompt (str) to be used for text generation.
        context = ""
        if blocks:
            for i, block in enumerate(blocks):
                context += f'Context #{i+1}: """{block}"""\n\n'
            context = self.limit_tokens(context, 2000)

        if mode == "creative":
            raise NotImplementedError

        elif mode == "balanced":
            system = '''You are a helpful assistant.'''
            user_prompt = '''You help users by answering questions and providing information about AI Alignment and AI Safety. You are extremely knowledgeable, yet you know the limits of your knowledge. Answer the questions as truthfully as possible using the provided context blocks, and if the answer is not contained within them, say, "I don't know." You can also ask the user questions to clarify their query.'''
            
            messages = [
                {"role": "system", "content": system},
                {"role": "user", "content": f"{user_prompt}\n\n{context}\n\nQuestion: {question}"}
            ]
        
        elif mode == "precise":
            raise NotImplementedError

        elif mode == "HyDE":
            assistant_prompt = "You are a helpful assistant, and you help users by answering questions and providing information about AI Alignment and AI Safety, on which you are extremely knowledgeable. Answer the user's question even if you are not certain of the answer; it is supremely important that you do attempt to offer an answer related to the user's query."
            messages = [
                {"role": "system", "content": assistant_prompt},
                {"role": "user", "content": question},
            ]
            
        else:
            raise ValueError("Mode must be one of 'balanced', 'precise', 'creative', or 'HyDE'.")
        
        return messages
    
    def answer_question(self, question: str, blocks: List[str]) -> str:
        # Receives a question (str) and a list of blocks and returns an answer (str) to the question.
        messages = self.construct_messages(question, blocks, mode="balanced")
        answer = openai.ChatCompletion.create(
            model=COMPLETIONS_MODEL, 
            messages=messages
        )
        return answer["choices"][0]["message"]["content"]
    
    def search_and_answer(self, question: str, k: int=10, HyDE: bool=False) -> str:
        # Receives a question (str) and returns an answer (str) to the question.
        if HyDE:
            messages = self.construct_messages(question, mode="HyDE")
            hyde_completion = openai.ChatCompletion.create(
                model=COMPLETIONS_MODEL, 
                messages=messages
            )
            top_k = self.get_top_k(f"{question}\n{hyde_completion}", k)
        else:
            top_k = self.get_top_k(question, k)
        answer = self.answer_question(question, top_k)
        return answer, top_k# , sources

In [156]:
k = 6
AS = AlignmentSearch(dataset=dataset)
query = """Say I don't know."""
# top_k_sources = AS.get_top_k(query, k)
answer, top_k_sources = AS.search_and_answer(query, k)#, HyDE=True)
print(answer)

Embedding created for query: Recognizing Human Actions in Data
IRL is a promising approach to learning human values in part because of the easy availability of data. For supervised learning, humans need to produce many labeled instances specialized for a task. IRL, by contrast, is an unsupervised/semi-supervised approach where any record of human behavior is a potential data source. Facebook’s logs of user behavior provide trillions of data-points. YouTube videos, history books, and literature are a trove of data on human behavior in both actual and imagined scenarios. However, while there is lots of existing data that is informative about human preferences, we argue that exploiting this data for IRL will be a difficult, complex task with current techniques.


KeyboardInterrupt: 

In [153]:
for source in top_k_sources:
    print(source)
    print("\n"*4)

Potential impacts of AI range from the immediate and particular to the vast and transformative. While most current scholarly and policy commentary on AI impacts addresses near-term advances and concerns, popular accounts are dominated by vivid scenarios of existential threats to human survival or autonomy, often inspired by fictional accounts in which AI has advanced to general super-intelligence, independent volition, or some other landmark of capabilities equivalent to exceeding those of humans. Expert opinions about the likelihood and timing of such extreme advances vary widely. 2 Yet it is also increasingly clear that such extreme advances in capability are not necessary for AI to have transformative societal impacts—for good or ill, or more likely for both—including the prospect of severe disruptions. Efforts to manage societal impacts of technology always face deep uncertainties, both about trends in technical capabilities and about how they will be used in social context. These 

In [ ]:
answer

('Context #3 suggests that the limitations on the number of universes that can be real are not yet clear. However, it is suggested that the universe may contain causal bubbles that allow some pieces of spacetime to survive superintelligences appearing in other pieces of spacetime, while the absence of causal bubbles makes it that a superintelligence or collection of superintelligences probably eventually takes over everything.',
 [' 2021-11-20\n\n ## no room above paperclips\n\n (edit: see also [*yes room above paperclips? *](https://carado. moe/above-paperclips-2.\n\n when presented with the idea of a [paperclip-maximizing unaligned superintelligence](https://en. wikipedia.org/wiki/Instrumental_convergence#Paperclip_maximizer), people sometimes mention the possibility that sure, the universe gets tiled with paperclips, but maybe there\'s [slack](https://thezvi. wordpress.com/2017/09/30/slack/) in how paperclips are arranged, and that maybe nice things can exist again "above" paperclip

## Tests

### Comparing quantity of articles/words/tokens/characters in the official dataset vs. what we have

In [158]:
num_articles_truth = {
    'https://aipulse.org': 23,
    'ebook': 23,
    'https://qualiacomputing.com': 278,
    'alignment forum': 2138,
    'lesswrong': 28252 + 227,
    'manual': "?",
    'arxiv': 707 + 1679 + 1000 + 4621,
    'https://deepmindsafetyresearch.medium.com/': 10,
    'waitbutwhy.com': 2,
    'GitHub': "?",
    'https://aiimpacts.org': 227,
    'arbital.com': 223,
    'carado.moe': 59,
    'nonarxiv_papers': "?",
    'https://vkrakovna.wordpress.com': 43,
    'https://jsteinhardt.wordpress.com': 39,
    'audio-transcripts': 25 + 12,
    'https://intelligence.org': 479,
    'youtube': 457,
    'reports': "?",
    'https://aisafety.camp': 8,
    'curriculum': "?",
    'https://www.yudkowsky.net': 23,
    'distill': 49,
    'total': 2138+28252+707+1679+1000+4621+23+227+23+8+59+111+10+17+7+479+39+278+43+2+23+420+323+49+457+25+12+223+227+132    
}
word_count_truth = 53_550_146
char_count_truth = 351_767_163

# Print table. First row has Truth and Empirical findings.
print(f"{'Source':<20} {'Truth':<10} {'Empirical':<10} {'Difference':<10}")
for source in dataset.articles_count:
    try:
        print(f"{source[:20]:<20} {num_articles_truth[source]:<10} {dataset.articles_count[source]:<10} {num_articles_truth[source] - dataset.articles_count[source]:<10}")
    except TypeError:
        print(f"{source[:20]:<20} {num_articles_truth[source]:<10} {dataset.articles_count[source]:<10} {'UNKNOWN':<10}")

# Compare true and empirical word counts and character counts
print(f"\n{'':<20} {'Truth':<10} {'Empirical':<10} {'Difference':<10}")
print(f"{'Word Count':<20} {word_count_truth:<10} {dataset.total_word_count:<10} {word_count_truth - dataset.total_word_count:<10}")
print(f"{'Character Count':<20} {char_count_truth:<10} {dataset.total_char_count:<10} {char_count_truth - dataset.total_char_count:<10}")

Source               Truth      Empirical  Difference
https://aipulse.org  23         23         0         
manual               ?          1          UNKNOWN   
waitbutwhy.com       2          2          0         
https://vkrakovna.wo 43         43         0         
https://jsteinhardt. 39         39         0         
https://aisafety.cam 8          8          0         
curriculum           ?          1          UNKNOWN   
https://www.yudkowsk 23         23         0         

                     Truth      Empirical  Difference
Word Count           53550146   335374     53214772  
Character Count      351767163  2096402    349670761 


### Embeeding time estimation

In [ ]:
# Define a helper function that takes in a single string and outputs a single d-dimensional vector
def get_embedding(text):
  # Use the embeddings OpenAI API endpoint to get an embedding for the text
  result = openai.Embedding.create(model=EMBEDDING_MODEL, input=text)
  # Convert the response to a numpy array and return it
  return result["data"][0]["embedding"]

# Define a function that takes in a list of strings and outputs a numpy matrix of embeddings
def get_embeddings(texts):
  embeddings = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_embedding, text) for text in texts]
    for future in concurrent.futures.as_completed(futures):
      embeddings.append(future.result())
  return np.vstack(embeddings)

def get_embeddings_not_parallel(texts):
    embeddings = np.array([get_embedding(text) for text in texts])
    return embeddings

In [ ]:
# Define a list of texts to be embedded
texts = ["Hello world!"] * 100

# Regular method
start = time.time()
embeddings_1 = get_embeddings_not_parallel(texts)
end = time.time()
print(f"Regular method: {end - start}s")

# Parallel method
start = time.time()
embeddings_2 = get_embeddings(texts)
end = time.time()
print(f"Parallel method: {end - start}s")

Regular method: 23.137897491455078
Parallel method: 1.7662100791931152
(100, 1536)
(100, 1536)


Conclusion: async method is ~10x faster than sync method

### Streaming ChatGPT response

In [ ]:
import asyncio
import config
import openai
openai.api_key = config.OPENAI_API_KEY

In [2]:
import asyncio

async def test_stream(question: str):
  assistant_prompt = "You are a helpful assistant, and you help users by answering questions and providing information about AI Alignment, on which you are extremely knowledgeable. Answer the user's question even if you are not certain of the answer; it is supremely important that you do attempt to offer an answer related to the user's query."
  
  messages = [
    {"role": "system", "content": assistant_prompt},
    {"role": "user", "content": question},
  ]
  async for part in await openai.ChatCompletion.acreate(
    model=COMPLETIONS_MODEL,
    messages=messages,
    stream=True
  ):
    finish_reason = part["choices"][0]["finish_reason"]
    if "content" in part["choices"][0]["delta"]:
      content = part["choices"][0]["delta"]["content"]
      print(content, end="")
    elif finish_reason:
      print(finish_reason)

if __name__ == '__main__':
  question = "What is the Natural Abstraction Hypothesis?"
  asyncio.run(test_stream(question))

RuntimeError: asyncio.run() cannot be called from a running event loop

### Tiktoken-related tests

In [ ]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
encoding.encode("tiktoken is great!")

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

a = num_tokens_from_string("tiktoken is great!", "cl100k_base")
print(a)
# Output: 6

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    except AttributeError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
b = num_tokens_from_messages([{"name": "user", "content": "hello"}], model="gpt-3.5-turbo-0301")
print(b)
# Output: 7

In [ ]:
import tiktoken

def limit_tokens(text: str, max_tokens: int, encoding_name: str = "cl100k_base") -> str:
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)[:max_tokens]
    return encoding.decode(tokens)

# Example usage
input_text = "tiktoken is a great library to work with tokens in text!"
limited_text = limit_tokens(input_text, 1000)
print(limited_text)

tiktoken is a great library to work with tokens in text!


In [5]:
import pinecone
import config
PINECONE_API_KEY = config.PINECONE_API_KEY

pinecone.init(api_key=PINECONE_API_KEY, environment="us-central1-gcp")

In [6]:

pinecone.create_index("quickstart", dimension=8, metric="euclidean", pod_type="p1")
pinecone.list_indexes()
# Returns:
# ['quickstart']

['quickstart']

In [7]:
index = pinecone.Index("quickstart")

In [8]:
# Upsert sample data (5 8-dimensional vectors)
index.upsert([
    ("A", [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
    ("B", [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]),
    ("C", [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]),
    ("D", [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]),
    ("E", [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
])

{'upserted_count': 5}

In [9]:
index.describe_index_stats()
# Returns:
# {'dimension': 8, 'index_fullness': 0.0, 'namespaces': {'': {'vector_count': 5}}}

{'dimension': 8,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [10]:
index.query(
  vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
  top_k=3,
  include_values=True
)
# Returns:
# {'matches': [{'id': 'C',
#               'score': 0.0,
#               'values': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
#              {'id': 'D',
#               'score': 0.0799999237,
#               'values': [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]},
#              {'id': 'B',
#               'score': 0.0800000429,
#               'values': [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]}],
#  'namespace': ''}

{'matches': [{'id': 'C',
              'score': 0.0,
              'values': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
             {'id': 'D',
              'score': 0.0799999237,
              'values': [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]},
             {'id': 'B',
              'score': 0.0800000429,
              'values': [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]}],
 'namespace': ''}

In [ ]:
pinecone.delete_index("quickstart")